[![imagenes/pythonista.png](imagenes/pythonista.png)](https://pythonista.io)

# Seguridad y autenticación básicas.

*Django* cuenta desde su instalación con un sistema simple de:

* Gestión de usuarios.
* Gestión de grupos.
* Autenticación básica.
* Gestión de sesiones.

La documentación correspondiente puede ser consultada en la siguiente liga:

https://docs.djangoproject.com/en/3.2/topics/auth/

**Nota:** Los alcances de este curso sólo cubren los temas de gestión básica de usuarios y autenticación simple.

## Aplicaciones y middleware asociados.

Las aplicaciones instaladas para la gestión de usuarios y permisos son:
* ```django.contrib.auth``` para la gestión de usuarios y grupos.
* ```django.contrib.contenttypes``` para asignar permisos a los modelos que cree el desarrollador.

In [ ]:
from tutorial.tutorial import settings

In [ ]:
settings.INSTALLED_APPS

El middleware que utiliza *Django* por efecto es:

* ```django.contrib.sessions.middleware.SessionMiddleware```, el cual permite realizar la autenticación y autorización de acceso.
* ```django.contrib.auth.middleware.AuthenticationMiddleware```, el cual gestiona las sesiones.

In [ ]:
settings.MIDDLEWARE

##  El modelo  ```django.contrib.auth.models.User```.  

Los usuarios y grupos en *Django* se crean mediante modelos predeterminados.

* La clase ```django.contrib.auth.models.User``` es una subclase de  ```django.db.models.Model``` y corresponde al modelo básico de un usuario de *Django*. A partir de este momento se hará referencia a esta clase como ```User```.

### El atributo de clase ```User.objects```.

Al ser una subclase de ```django.db.models.Model```, el modelo ```User``` cuenta con el atributo de clase ```objects```, el cual puede realizar operaciones de consulta, altas y bajas de instancias mediante métodos como:

* ```all()```.
* ```get()```.
* ```filter()```.

#### El método ```User.objects.create_user()```.

Este método permite crear la cuenta de un usuario y guardarla en la base de datos.

```
 User.objects.create_user('<nombre de usuario>', '<correo electrónico>', '<contraseña>')
```

### Atributos de las instancias de  ```User```.

Las instancias del modelo ```User``` cuenta con los siguientes atributos:

* ```id```, el cual corresponde a un identificador numérico que *Django* le otorga al modelo cuando es instanciado.
* ```username```, el cual corresponde al nombre de usuario. Este atributo es obligatorio para crear al modelo.
* ```password```, el cual corresponde a la contraseña del usuario. Este atributo es obligatorio para crear al modelo.
* ```first_name```, el cual corresponde al nombre del usuario.
* ```last_name```, el cual corresponde al apellido del usuario.
* ```email```, el cual corresponde a la dirección de correo electrónico del usuario.
* ```is_superuser``` es un valor de tipo ```bool``` que indica si el usuario es in superusuario.
* ```is_staff```  es un valor de tipo ```bool``` que indica si el usuario es parte del "staff" de la organización.
* ```is_active``` es un valor de tipo ```bool``` que indica si el usuario está activo.
* ```last_login``` es un valor de tipo ```datetime.datetime```con la fecha y hora del último acceso del usuario.
* ```date_joined``` es un valor de tipo ```datetime.datetime```con la fecha y hora en la que el usuario fue creado.

## Ejemplo ilustrativo de creación de un suario desde  el shell.

``` ipython
In [1]: from django.contrib.auth.models import User

In [2]: User.objects.all()
Out[2]: <QuerySet []>

In [3]: User.objects.create_user('josech', 'josech@gmail.com', '0p3n5t4ck')
Out[3]: <User: josech>

In [4]: User.objects.all()
Out[4]: <QuerySet [<User: josech>]>


In [5]: usuario = User.objects.all()[0]

In [5]: usuario
Out[5]: <User: josech>

In [6]: usuario.is_superuser
Out[6]: False

In [7]: usuario.is_superuser=True

```

## Creación de un superusuario con el script ```manage.py```.

Para poder crear unsuperuisuario se utiliza el siguiente comando desde una terminal localziada en el directorio del proyecto.

```
python manage.py createsuperuser --email="<correo electronico>" --user="<usuario>"
```

**Ejemplo:**
```
python manage.py createsuperuser --email="falso@pythonista.io" --user="admin"
```

## Tablas asociadas a usuarios y grupos en la base de datos.

La gestión de usuarios está ligada a varias tablas de la base de datos asociada a *Django*, la cuales fueron creadas al ejecutar el comando ```manage.py migrate```.

Estas tablas tienen el prefijo ```auth_```:

* ```auth_user```.
* ```auth_group```.
* ```auth_permission```.
* ```auth_group_permissions```.
* ```auth_user_groups```.
* ```auth_user_user_permissions```.


In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///tutorial/db.sqlite3

In [ ]:
%%sql sqlite:///tutorial/db.sqlite3
SELECT name FROM sqlite_master WHERE type = "table"

In [ ]:
%sql SELECT * FROM auth_user

## El módulo ```django.contrib.auth.urls```.

Este módulo define los siguientes patrones de *URL* predefinidos para gestionar los accesos de un usuario:

* ```'login/'``` ingreso a la sesión de un usuario. 
* ```'logout/'``` salida de la sesión de un usuario.
* ```'password_change/'```, la cual permite cambiar la contraseña de un usuario.
* ```'password_reset/'```, la cual permite recuperar una contraseña.
* ```'reset/<uid64>/<token>'```, el cual permite reiniciar a un usuario.
* ```'password_change/done/'```.
* ```'password_reset/done/'```.
* ```'reset/done/'```.

Para que estos patrones puedan ser accedidos es necesario incluir un patrón en el objeto ```urlpatterns``` del script ```urls.py``` del proyecto de la siguiente manera:

```
 path('<ruta base>', include('django.contrib.auth.urls'))
```

* Donde ```<ruta base>``` corresponde a la ruta desde la cual se accederá a cada *URL* definida por ```django.contrib.auth.urls```. 

Con excepción de ```'login/'``` cada función de vista relacionada con cada patrón cuenta con una plantilla predefinida. Sin embargo, es posible crear plantillas a la medida.

Pro defecto las *URLs* de este módulo buscarán las plantillas correspondientes en el directorio```templates/registration/``` el cual se encuentra a su vez en el directorio definido para las plantillas del proyecto.

### Configuración de la función de la *URL* ```login/```

En el caso de la *URL* ligada a la regla ```'login/'```, es necesario crear una plantilla en la ruta ```registration/login.html```, la cual permita enviar  el usuario y la contraseña.

Un ejemplo de esta plantilla puede ser consultado en https://docs.djangoproject.com/en/3.2/topics/auth/default/#all-authentication-views.

### Redireccionamiento en caso de un ingreso correcto.

Cuando las credenciales ingresadas y enviadas son correctas, *Django* redirigirá al navegador a la ruta ```'/accounts/profile/'```. 

Para indicar un redireccionamiento a una *URL* distinta es necesario definirla con el nombre ```LOGIN_REDIRECT_URL``` en el script ```settings.py``` del proyecto.

```
LOGIN_REDIRECT_URL = '<ruta>'
```

## Ejemplo ilustrativo.

### Definición del patrón de *URLs*.

El archivo ```src/21/urls.py``` contiene la siguiente definición de  ```urlpatterns``` ligando al módulo ``` django.contrib.auth.urls``` con la ruta ```usuarios/```.

```python
urlpatterns = [path('admin/', admin.site.urls), 
               path('main/', include('main.urls')),
               path('api/', include('api.urls')),
               path('accounts/', include('django.contrib.auth.urls')),
              ]
```

* Se sustituirá el script ```tutorial/tutorial/urls.py``` con ```src/22/urls.py```.

* Para las platafomas Linux y MacOS X.

In [ ]:
!cp src/21/urls.py tutorial/tutorial/urls.py

* Para la plataforma Windows.

In [ ]:
!copy src\21\urls.py tutorial\tutorial\urls.py

* La siguiente celda desplegará el archivo sustituido.

In [ ]:
%pycat tutorial/tutorial/urls.py

### Creación del subdirectorio ```registration/```.

* La siguiente celda creará el directorio ```registration/``` dentro del directorio ```tutorial/templates```.

In [ ]:
%mkdir tutorial/templates/registration

In [ ]:
%mkdir tutorial\templates\registration

### La plantilla ```registration/login.html```.

La plantilla  ```src/22/login.html``` contiene el siguiente código:

``` html
{% extends "base.html" %}

{% block cuerpo %}

{% if form.errors %}
  <p>Ingresó una contraseña incorrecta. Vuelva a intentar.</p>
{% endif %}

{% if next %}
  {% if user.is_authenticated %}
    <p>Su cuenta no tiene acceso a esta página. Vuelva a intentar con un usuario autorizado.</p>
  {% else %}
    <p>Por favor ingrese con usuario con los permisos adecuados.</p>
  {% endif %}
{% endif %}

<form method="post" action="{% url 'login' %}">
{% csrf_token %}
<table>

<tr>
  <td>{{ form.username.label_tag }}</td>
  <td>{{ form.username }}</td>
</tr>

<tr>
  <td>{{ form.password.label_tag }}</td>
  <td>{{ form.password }}</td>
</tr>
</table>

<input type="submit" value="login" />
<input type="hidden" name="next" value="{{ next }}" />
</form>

{# Assumes you setup the password_reset view in your URLconf #}
<p><a href="{% url 'password_reset' %}">¿Perdió su contraseña?</a></p>

{% endblock %}
```

* A continuacion se copiará al script ```src/21/urls.py``` en el directorio ```tutorial/templates/registration/```.

* Para las platafomas Linux y MacOS X.

In [ ]:
%cp src/21/login.html tutorial/templates/registration/

* Para la platafoma Windows.

In [ ]:
%copy src\21\login.html tutorial\templates\registration\

* La siguiente celda despelgará el contenido del script ```tutorial/templates/registration/login.html```

In [ ]:
%pycat tutorial/templates/registration/login.html

### Configuración del redireccionamiento de ingreso exitoso.

El script ```src/21/settings.py``` define a ```/api/``` como la ruta a la que el navegador será redirigido en caso de que el acceso sea exitoso.

``` python
LOGIN_REDIRECT_URL = '/api/'
```

* A continuación se sustituirá al script ```tutorial/tutorial/settings.py``` con ```src/21/settings.py```.

* Para las platafomas Linux y MacOS X.

In [ ]:
%cp src/21/settings.py tutorial/tutorial/settings.py

* Para la platafoma Linux.

In [ ]:
%copy src\21\settings.py tutorial\tutorial\settings.py

La siguiente celda mostrará el resultado de la sustitución en ```tutorial/tutorial/settings.py```.

In [ ]:
%pycat tutorial/tutorial/settings.py

## Arranque desde una terminal.

* Desde una terminal diríjase al directorio ```tutorial```, en el cual se encuentra el script ```manage.py```.
* Ejecute el siguiente comando:


```
python manage.py runserver 0.0.0.0:8000
```

**Nota:** 
Es necesario que el firewall de su equipo esté configurado para transmitir desde el puerto ```8000```. 

## Resultados.

http://localhost:8000/accounts/login/

<p style="text-align: center"><a rel="license" 
                                 href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2021.</p>